<a href="https://colab.research.google.com/github/degru82/programmers-de/blob/master/dataengineering_w2_mau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로그래머스 데이터엔지니어링 2주차

1. db-conn 함수의 작성
1. MAU 계산을 SQL을 사용해서
1. MAU 계산을 Pandas를 사용해서

----
## 1. 먼저 db-conn 함수를 작성한다

In [1]:
import psycopg2

def connect_to_redshift(
    user: str, pwd: str,
    host: str="grepp-data.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com",
    port: int=5439, dbname: str='dev',
)->tuple:
    """
    레드시프트 데이터베이스에 연결하고 정보를 리턴합니다

    returns (connection, cursor)

    inputs
    - user: username
    - pwd: password
    - host: redshift url (default: grepp-data.blahblah)
    - port: port number (default: 5439)
    - dbname: database name (default: dev)
    """

    info = \
        f'dbname={dbname} user={user} host={host} password={pwd} port={port}'
    conn = psycopg2.connect(info)

    return conn, conn.cursor()


connect_to_redshift?

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# 실행환경이 콘솔이었다면, 환경변수에 지정했겠지만,
# 주피터 노트북에서 실행할 예정이므로 ID/PWD를 입력받습니다

user = input('Type Username: ')
pwd = input('Type Password: ')

conn, cursor = connect_to_redshift(user, pwd)
conn, cursor

----
## 2. MAU 계산을 SQL을 사용해서

In [3]:
query = '''
SELECT 
    COUNT(DISTINCT USC.userid)
    , EXTRACT(YEAR FROM ST.ts) AS YYYY
    , EXTRACT(MONTH FROM ST.ts) AS MM
FROM raw_data.user_session_channel USC

JOIN raw_data.session_timestamp ST
ON USC.sessionid=ST.sessionid

GROUP BY 2, 3
ORDER BY 2, 3
'''

cursor.execute(query)
result = cursor.fetchall()

In [4]:
for num_users, year, month in result:

    print(f'{num_users} MAUs on {year}-{month:02}')

281 MAUs on 2019-05
459 MAUs on 2019-06
623 MAUs on 2019-07
662 MAUs on 2019-08
639 MAUs on 2019-09
763 MAUs on 2019-10
721 MAUs on 2019-11


----
## 3. MAU 계산을 Pandas를 사용해서
- 두 개의 테이블을 읽어와서
- 합쳐서 하나의 테이블을 만들고 (join)
- 연도와 월 정보를 추출해서 새로운 컬럼을 만든 뒤,
- 해당 연월에 유니크한 사용자 아이디 개수를 구한다

In [5]:
import pandas.io.sql as sqlio

query = "select * from raw_data.user_session_channel"
df_usc = sqlio.read_sql(query, conn)
df_usc.head()



,userid,sessionid,channel
0,184,c41dd99a69df04044aa4e33ece9c9249,Naver
1,80,fdc0eb412a84fa549afe68373d9087e9,Organic
2,251,0a54b19a13b6712dc04d1b49215423d8,Facebook
3,264,a914ecef9c12ffdb9bede64bb703d877,Google
4,744,05ae14d7ae387b93370d142d82220f1b,Facebook


In [6]:
query = "select * from raw_data.session_timestamp"
df_st = sqlio.read_sql(query, conn)
df_st.head()

,sessionid,ts
0,c41dd99a69df04044aa4e33ece9c9249,2019-05-01 00:35:59.897
1,fdc0eb412a84fa549afe68373d9087e9,2019-05-01 02:53:48.503
2,0a54b19a13b6712dc04d1b49215423d8,2019-05-01 12:18:26.637
3,a914ecef9c12ffdb9bede64bb703d877,2019-05-01 13:41:29.267
4,05ae14d7ae387b93370d142d82220f1b,2019-05-01 14:17:54.083


In [7]:
import pandas as pd

df_merged = pd.merge(df_usc, df_st, how='outer', on='sessionid')
df_merged.tail()

,userid,sessionid,channel,ts
101515,45,691f008d42e361433415db5a7773d697,Instagram,2019-11-30 22:54:15
101516,2181,60d1f51c17418bb12b9bf94d289c7204,Google,2019-11-30 23:25:11
101517,2337,9cfaefd1e81f637fad6330ff16eb1f39,Organic,2019-11-30 23:46:39
101518,1285,f06a4ccd032e58412b6452b2536ecbf2,Youtube,2019-11-29 15:10:30
101519,2271,f551359ab7f6d759eb1880f554e5e815,Naver,2019-11-29 15:06:44


In [8]:
yyyymm = df_merged['ts'].apply(lambda x: x.strftime('%Y%m'))
df_merged['yyyymm'] = yyyymm
df_merged.head()

,userid,sessionid,channel,ts,yyyymm
0,184,c41dd99a69df04044aa4e33ece9c9249,Naver,2019-05-01 00:35:59.897,201905
1,80,fdc0eb412a84fa549afe68373d9087e9,Organic,2019-05-01 02:53:48.503,201905
2,251,0a54b19a13b6712dc04d1b49215423d8,Facebook,2019-05-01 12:18:26.637,201905
3,264,a914ecef9c12ffdb9bede64bb703d877,Google,2019-05-01 13:41:29.267,201905
4,744,05ae14d7ae387b93370d142d82220f1b,Facebook,2019-05-01 14:17:54.083,201905


In [9]:
yyyymm.unique()

array(['201905', '201906', '201907', '201908', '201909', '201910',
       '201911'], dtype=object)

In [10]:
mau_yyyymm = {}

for yyyymm in yyyymm.unique():
    on_yyyymm = (df_merged['yyyymm'] == yyyymm)
    num_users = len(df_merged[on_yyyymm]['userid'].unique())
    mau_yyyymm[yyyymm] = num_users

mau_yyyymm

{'201905': 281,
 '201906': 459,
 '201907': 623,
 '201908': 662,
 '201909': 639,
 '201910': 763,
 '201911': 721}

In [11]:
conn.close()